In [1]:
import os
import numpy as np
#from prototype import prototype
#import scipy.spatial.distance as ssd
#from dissimilarity import dissimilarity
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import cifar10
from PIL import Image

2023-08-21 09:28:18.025607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#if you want to equalize fashion_mnist,
#mnist=fashion_mnist

train=mnist.load_data()[0][0]
train_label=mnist.load_data()[0][1]
x_test=mnist.load_data()[1][0]
y_test=mnist.load_data()[1][1]
y_test=y_test.reshape(len(y_test))

Defined histograms

In [ ]:
for value in range(8):
    entropy=value+1
    area=2**entropy
    Q=int(train.shape[1]*train.shape[2]/area)
    mod=train.shape[1]*train.shape[2]%area
    pixel_set=[]
    for n in range(area):
        if n < mod:
            for i in range(Q+1):
                pixel_set.append(n)
        else:
            for i in range(Q):
                pixel_set.append(n)


    equalized=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
    a=train.reshape(-1,train.shape[1]*train.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set)):
        equalized[ind2==i]=int(pixel_set[i])
    equalized=equalized.reshape(train.shape[0],train.shape[1],train.shape[2])
    equalized_test=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
    a=x_test.reshape(-1,x_test.shape[1]*x_test.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set)):
        equalized_test[ind2==i]=int(pixel_set[i])
    equalized_test=equalized_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2])
    makefile(equalized,"Data/MNIST_edit/x_train"+str(entropy)+".pkl")
    makefile(equalized_test,"Data/MNIST_edit/x_test"+str(entropy)+".pkl")

Average for classes

In [ ]:

for label in range(10):
    print(label)
    x_train=train[train_label==label]
    pixel_list=dict()
    for n in range(256):
        pixel_list[n]=0
    for data in x_train:
        for raw in data:
            for pixel in raw:
                pixel_list[pixel]+=1
    hist_list=dict()
    for num in pixel_list:
        hist_list[num]=pixel_list[num]/len(x_train)
    hist_num=dict()
    residual=0
    for num in hist_list:
        hist_num[num]=int(np.round(hist_list[num]+residual,0))
        residual=hist_list[num]+residual-int(np.round(hist_list[num]+residual,0))

        
    
    pixel_set=[]
    for num in hist_num:
        repeat=hist_num[num]
        for n in range(repeat):
            pixel_set.append(num)

    equalized=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
    a=train.reshape(-1,train.shape[1]*train.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set)):
        equalized[ind2==i]=int(pixel_set[i])
    equalized=equalized.reshape(train.shape[0],train.shape[1],train.shape[2])
    equalized_test=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
    a=x_test.reshape(-1,x_test.shape[1]*x_test.shape[2])
    ind=np.argsort(a,axis=1,kind="mergesort")
    ind2=np.argsort(ind,axis=1)
    for i in range(len(pixel_set)):
        equalized_test[ind2==i]=int(pixel_set[i])
    equalized_test=equalized_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2])
    makefile(equalized,"Data/MNIST_edit/x_train_average_"+str(label)+".pkl")
    makefile(equalized_test,"Data/MNIST_edit/x_test_average_"+str(label)+".pkl")

In [ ]:
#average all
x_train=train
pixel_list=dict()
for n in range(256):
    pixel_list[n]=0
for data in x_train:
    for raw in data:
        for pixel in raw:
            pixel_list[pixel]+=1
hist_list=dict()
for num in pixel_list:
    hist_list[num]=pixel_list[num]/len(x_train)
hist_num=dict()
residual=0
for num in hist_list:
    hist_num[num]=int(np.round(hist_list[num]+residual,0))
    residual=hist_list[num]+residual-int(np.round(hist_list[num]+residual,0))



pixel_set=[]
for num in hist_num:
    repeat=hist_num[num]
    for n in range(repeat):
        pixel_set.append(num)

equalized=np.zeros([train.shape[0],train.shape[1]*train.shape[2]])
a=train.reshape(-1,train.shape[1]*train.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set)):
    equalized[ind2==i]=int(pixel_set[i])
equalized=equalized.reshape(train.shape[0],train.shape[1],train.shape[2])
equalized_test=np.zeros([x_test.shape[0],x_test.shape[1]*x_test.shape[2]])
a=x_test.reshape(-1,x_test.shape[1]*x_test.shape[2])
ind=np.argsort(a,axis=1,kind="mergesort")
ind2=np.argsort(ind,axis=1)
for i in range(len(pixel_set)):
    equalized_test[ind2==i]=int(pixel_set[i])
equalized_test=equalized_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2])
makefile(equalized,"Data/MNIST_edit/x_train_average_all.pkl")
makefile(equalized_test,"Data/MNIST_edit/x_test_average_all.pkl")